In [ ]:
import torch
from util import log_tensor

In [ ]:
channel_size = 3
batch_size = 2
input_size = 4
output_size = 2

channel_lbls = [f'channel {i}' for i in range(channel_size)]
ent_lbls = [f'ent {i}' for i in range(batch_size)]
emb_lbls = [f'emb {i}' for i in range(input_size)]
class_lbls = [f'class {i}' for i in range(output_size)]

my_tensor = torch.randn(batch_size, channel_size, input_size)
log_tensor(my_tensor, 'my_tensor', [ent_lbls, channel_lbls, emb_lbls])

weight = torch.randn(channel_size, input_size, output_size)
log_tensor(weight, 'weight', [channel_lbls, emb_lbls, class_lbls])

bias = torch.randn(channel_size, 1, output_size)
log_tensor(bias, 'bias', [channel_lbls, [''], class_lbls])

output = torch.bmm(my_tensor.transpose(0, 1), weight) + bias
log_tensor(output, 'output', [channel_lbls, ent_lbls, class_lbls])

In [ ]:
batch_size = 2
class_count = 3
emb_size = 4

ent_lbls = [f'ent {i}' for i in range(batch_size)]
class_lbls = [f'class {i}' for i in range(class_count)]
mix_lbls = [f'mix {i}' for i in range(class_count)]
emb_lbls = [f'emb {i}' for i in range(emb_size)]

# (batch_size, class_count, emb_size)
mixes_batch = torch.tensor([
    [[1., 0., 0., 0.], [0., 1., 0., 0.], [0., 0., 1., 0.]],
    [[-1., 0., 0., 0.], [0., -1., 0., 0.], [0., 0., -1., 0.]]
])
log_tensor(mixes_batch, 'mixes_batch', [ent_lbls, mix_lbls, emb_lbls])

# (class_count, emb_size)
weight = torch.randn(class_count, emb_size, requires_grad=True)
log_tensor(weight, 'weight', [class_lbls, emb_lbls])

# (class_count, 1)
bias = torch.randn(class_count, 1, requires_grad=True)
log_tensor(bias, 'bias', [class_lbls, ['']])

In [ ]:
criterion = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam([weight, bias], lr=0.1)

gt = torch.tensor([
    [1, 1, 1],
    [0, 0, 0]
])

for epoch in range(20):
    print(f'Epoch {epoch}')

    #
    # < mixes_batch  (batch_size, class_count, emb_size)
    # < weight       (class_count, emb_size)
    # < bias         (class_count, 1)
    # > logits       (batch_size, class_count)
    #

    logits = torch.bmm(mixes_batch.transpose(0, 1), weight.unsqueeze(-1)) + bias.unsqueeze(-1)
    logits = logits.transpose(0, 1).squeeze(-1)

    # (batch_size, class_count)
    softs = torch.nn.Sigmoid()(logits)
    log_tensor(softs, 'softs', [ent_lbls, class_lbls])

    log_tensor(gt, 'gt', [ent_lbls, class_lbls])

    loss = criterion(logits, gt.float())

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

In [ ]:
target = torch.randint(0, 10, (10,))
one_hot = torch.nn.functional.one_hot(target)

print(target)
print(one_hot)

gt0 = torch.tensor([
    [1, 1, 1],
    [0, 0, 0]
])

gt = torch.tensor([
    [[0, 1], [0, 1], [0, 1]],
    [[1, 0], [1, 0], [1, 0]]
])

torch.nn.functional.one_hot(gt0)
torch.argmax(gt, dim=-1)